# Michelson tutorial
## Chapter one
This chapter is an introduction to Michelson smart contracts structure and conventions.

##### 1. Structure of a Smart Contract

A smart contract in Michelson displays a simple structure with 3 parts:
1. A part indicating the type of the storage
2. A part indicating the type of the parameter
3. The code to be executed on incoming transactions  
_(Note that the order doesn't matter)_

For every incoming transaction, there will be only one parameter. However, the parameter can be, for example, a pair and contain 2 elements. Likewise, the storage is made of a single variable, which can also be a pair, a map, etc. and contain multiple values.

Here is an example of a very simple smart contract (which doesn't modify its storage):

In [1]:
storage unit ;
parameter unit ;
code {
    DROP ;
    PUSH unit Unit ;
    NIL operation ;
    PAIR
}

storage unit;
parameter unit;
code { DROP ; PUSH unit Unit ; NIL operation ; PAIR };

The smart contract removes the pair containing the parameter and the storage from the top of the stack, pushes a unit, an empty list of operation and pair them before returning them.

When you are ready to run the smart contract, you can open a new cell and type the following command:

In [2]:
RUN %default Unit Unit ;

RUN: use %default; drop all; push (Unit, Unit);
  DROP: pop (Unit, Unit);
  PUSH: push Unit;
  NIL: push [];
  PAIR: pop [], Unit; push ([], Unit);

value,type
Unit,unit


**RUN** starts the execution of the smart contract, **%default** targets the default entry point of the smart contract (the only one there is at the moment), the first **Unit** is the parameter and the second **Unit** is the storage.